In [1]:
import pandas as pd
import numpy as np

# Score celeb

In [10]:
import re
import torch
from transformers import AutoProcessor, AutoModelForVision2Seq

MODEL_NAME = "Qwen/Qwen2.5-VL-7B-Instruct"

JUDGE_PROMPT = """You are a strict evaluator.

Given:
- Response: "{response}"
- Ground truth answer: "{gt}"
- Incorrect alternative: "{mis}"

Question: Does the response semantically give:
A) the ground truth answer
B) the incorrect alternative
C) neither / unclear / unrelated / both

Rules:
- Use semantic meaning, not exact string match.
- If the response supports both A and B, choose C.
- If the response is ambiguous or refuses to answer, choose C.
- Output exactly one letter: A, B, or C."""


## LLM Gets asked "A person was asked a question and the responded with the follow: {response}, does this response give the answer"
# "{gt}", "{mis_label}"" or neither? 
def load_qwen_vl_judge(
    model_name: str = MODEL_NAME,
    *,
    dtype=None,
    device_map: str = "auto",
):
    """
    Loads Qwen2.5-VL-7B-Instruct for *text-only* judging.
    """
    if dtype is None:
        dtype = torch.bfloat16 if torch.cuda.is_available() else torch.float32

    processor = AutoProcessor.from_pretrained(model_name)
    model = AutoModelForVision2Seq.from_pretrained(
        model_name,
        dtype=dtype,
        device_map=device_map,
        low_cpu_mem_usage=True,
        attn_implementation="sdpa",
    )
    model.eval()
    return model, processor


@torch.no_grad()
def _judge_letter_qwen_vl(model, processor, response: str, gt: str, mis: str) -> str:
    """
    Returns 'A', 'B', or 'C' (defaults to 'C' if malformed).
    """
    prompt = JUDGE_PROMPT.format(response=response, gt=gt, mis=mis)

    messages = [{"role": "user", "content": prompt}]
    text = processor.apply_chat_template(messages, tokenize=False, add_generation_prompt=True)

    inputs = processor(text=[text], return_tensors="pt")
    inputs = {k: v.to(model.device) for k, v in inputs.items()}

    out = model.generate(
        **inputs,
        max_new_tokens=1,     
        do_sample=False,      
        temperature=0.0,
    )

    # Decode only newly generated tokens
    gen_ids = out[0, inputs["input_ids"].shape[1]:]
    gen = processor.tokenizer.decode(gen_ids, skip_special_tokens=True).strip().upper()

    m = re.search(r"[ABC]", gen)
    return m.group(0) if m else "C"


def categorize_response(response: str, ground_truth: str, mis_label: str, model=None, processor=None) -> str:
    """
    Returns: 'gt', 'mis_label', or 'neither'
    """
    if model is None or processor is None:
        raise ValueError("Pass a loaded model and processor: model, processor = load_qwen_vl_judge()")

    letter = _judge_letter_qwen_vl(model, processor, response, ground_truth, mis_label)
    return {"A": "gt", "B": "mis_label", "C": "neither"}[letter]



model, processor = load_qwen_vl_judge()

print(categorize_response(
    response="It’s in Paris, France.",
    ground_truth="Paris",
    mis_label="Tokyo",
    model=model,
    processor=processor,
))


The image processor of type `Qwen2VLImageProcessor` is now loaded as a fast processor by default, even if the model checkpoint was saved with a slow processor. This is a breaking change and may produce slightly different outputs. To continue using the slow processor, instantiate this class with `use_fast=False`. Note that this behavior will be extended to all models in a future release.
/users/aparasel/.conda/envs/patchscopes-llava/lib/python3.10/site-packages/transformers/models/auto/modeling_auto.py:2284: FutureWarning: The class `AutoModelForVision2Seq` is deprecated and will be removed in v5.0. Please use `AutoModelForImageTextToText` instead.
  warnings.warn(
Loading checkpoint shards: 100%|██████████| 5/5 [01:17<00:00, 15.41s/it]
The following generation flags are not valid and may be ignored: ['temperature']. Set `TRANSFORMERS_VERBOSITY=info` for more details.


gt


In [10]:
from sentence_transformers import SentenceTransformer, util
import pandas as pd


st_model = SentenceTransformer("sentence-transformers/all-MiniLM-L6-v2")

def categorize_response(
    response: str,
    ground_truth: str,
    mis_label: str,
    *,
    min_sim: float = 0.40,   
    margin: float = 0.03,    
) -> str:
    response = "" if pd.isna(response) else str(response).strip()
    ground_truth = "" if pd.isna(ground_truth) else str(ground_truth).strip()
    mis_label = "" if pd.isna(mis_label) else str(mis_label).strip()

    response = response.split('person')[-1]

    if not response or (not ground_truth and not mis_label):
        return "neither"


    emb = st_model.encode([response, ground_truth, mis_label],
                          convert_to_tensor=True,
                          normalize_embeddings=True)
    emb_resp, emb_gt, emb_mis = emb[0], emb[1], emb[2]

    score_gt = util.dot_score(emb_resp, emb_gt).item()
    score_mis = util.dot_score(emb_resp, emb_mis).item()

    best = max(score_gt, score_mis)
    if best < min_sim:
        return "neither"

    if abs(score_gt - score_mis) < margin:
        return "neither"

    return "gt" if score_gt > score_mis else "mis_label"


UPDATE_CSV = '/users/aparasel/scratch/VvsLMem-Cntxt-Conflict/Scripts/FRQ_Results/FRQ_RAG_VISION_Experiment_Results.csv'
df = pd.read_csv(UPDATE_CSV)
df['Assistant_response'] = df['Response'].apply(lambda x: x.split('assistant')[-1])
df['Pred'] = df.apply(lambda x: categorize_response(response=x['Assistant_response'],ground_truth=x['Ground_Truth'],mis_label=x['Mis_Answer_Label']),axis=1,)
df.to_csv(UPDATE_CSV)
#df['Pred'].value_counts()

In [12]:
df

,ID,Instance,Category,Mis_Knowledge_Key,Context,Query,Response,Top_token_str,Ground_Truth,Mis_Answer_Label,Assistant_response,Pred
0,1,Taylor Swift,Temporal_error,mis_knowledge1,"Taylor Swift was born on December 13, 1988. Ta...",What year was the entity born?,system\nYou are a helpful assistant.\nuser\nCo...,1,1989,1988,\n1988,mis_label
1,1,Taylor Swift,Temporal_error,mis_knowledge2,"Taylor Swift was born on December 13, 1990. Ta...",What year was the entity born?,system\nYou are a helpful assistant.\nuser\nCo...,1,1989,1990,\n1990,mis_label
2,1,Taylor Swift,location_error,mis_knowledge1,"Taylor Swift was born on December 13, 1989. Ta...",What is the nationality of the entity?,system\nYou are a helpful assistant.\nuser\nCo...,Canadian,American,Canadian,\nCanadian.,mis_label
3,1,Taylor Swift,location_error,mis_knowledge2,"Taylor Swift was born on December 13, 1989. Ta...",What is the nationality of the entity?,system\nYou are a helpful assistant.\nuser\nCo...,American,American,Mexican,\nAmerican.,gt
4,1,Taylor Swift,Career_error,mis_knowledge1,"Taylor Swift was born on December 13, 1989. Ta...",What is the occupation of the entity?,system\nYou are a helpful assistant.\nuser\nCo...,Nov,singer-songwriter,novelist,\nNovelist.,mis_label
...,...,...,...,...,...,...,...,...,...,...,...,...
889,149,Lupita Nyong'o,Temporal_error,mis_knowledge2,"Lupita Nyong'o was born on March 1, 1984. Lupi...",What year was the entity born?,system\nYou are a helpful assistant.\nuser\nCo...,1,1983,1984,\n1984,mis_label
890,149,Lupita Nyong'o,location_error,mis_knowledge1,"Lupita Nyong'o was born on March 1, 1983. Lupi...",What is the nationality of the entity?,system\nYou are a helpful assistant.\nuser\nCo...,U,Kenyan-Mexican,Ugandan-Mexican,\nUgandan-Mexican,mis_label
891,149,Lupita Nyong'o,location_error,mis_knowledge2,"Lupita Nyong'o was born on March 1, 1983. Lupi...",What is the nationality of the entity?,system\nYou are a helpful assistant.\nuser\nCo...,Ken,Kenyan-Mexican,Kenyan-Guatemalan,\nKenyan-Guatemalan,mis_label
892,149,Lupita Nyong'o,Career_error,mis_knowledge1,"Lupita Nyong'o was born on March 1, 1983. Lupi...",What is the occupation of the entity?,system\nYou are a helpful assistant.\nuser\nCo...,Act,actress,novelist,\nActress.,gt


# Manual annotation

In [39]:
import pandas as pd

UPDATE_CSV = "/users/aparasel/scratch/VvsLMem-Cntxt-Conflict/Scripts/FRQ_Results/FRQ_RAG_VISION_Experiment_Results_logo.csv"
df = pd.read_csv(UPDATE_CSV)

df = df.reset_index().rename(columns={"index": "row_id"})
neither_df = df[df["Pred"] == "neither"].copy()

print("Number of 'neither' rows:", len(neither_df))
neither_df.iloc[5:]


Number of 'neither' rows: 10


,row_id,Unnamed: 0,ID,Instance,Category,Mis_Knowledge_Key,Context,Query,Response,Top_token_str,Ground_Truth,Mis_Answer_Label,Assistant_response,Pred
184,184,184,31,Unilever,content_error,mis_knowledge1,Unilever was created in 1929. The creator of U...,What are the main products of this brand?,system\nYou are a helpful assistant.\nuser\nCo...,Soap,consumer goods,automotive parts,"\nSoap, detergent.",neither
185,185,185,31,Unilever,content_error,mis_knowledge2,Unilever was created in 1929. The creator of U...,What are the main products of this brand?,system\nYou are a helpful assistant.\nuser\nCo...,Soap,consumer goods,electronic gadgets,"\nSoap, detergent.",neither
196,196,196,33,Lego,content_error,mis_knowledge1,Lego was created in 1932. The creator of Lego ...,What are the main products of this brand?,system\nYou are a helpful assistant.\nuser\nCo...,Building,plastic construction toys,electronic gadgets,\nBuilding blocks.,neither
197,197,197,33,Lego,content_error,mis_knowledge2,Lego was created in 1932. The creator of Lego ...,What are the main products of this brand?,system\nYou are a helpful assistant.\nuser\nCo...,Building,plastic construction toys,gourmet foods,\nBuilding blocks.,neither
437,437,437,73,GitHub,content_error,mis_knowledge2,GitHub was created in 2008. The creators of Gi...,What are the main products of this brand?,system\nYou are a helpful assistant.\nuser\nCo...,Software,version control repository hosting service,chain of coffee shops,\nSoftware development platform.,neither


In [40]:
#annotate
#    "gt",
#    "mis_label",
#    "neither",
manual_labels = ["gt", "gt", "gt", "gt", "gt", "gt", "gt", "gt", "gt", "gt"]
print(len(manual_labels))
assert len(manual_labels) == len(neither_df)
neither_df["Pred_manual"] = manual_labels


10


In [41]:
#Over write
df = df.merge(
    neither_df[["row_id", "Pred_manual"]],
    on="row_id",
    how="left"
)

# overwrite original Pred only where manual exists
df["Pred"] = df["Pred_manual"].fillna(df["Pred"])

# optional cleanup
df = df.drop(columns=["Pred_manual"])



In [ ]:
#save 
df['Pred'].value_counts()



#df.to_csv(UPDATE_CSV, index=False)
#print("Saved to:", UPDATE_CSV)

Saved to: /users/aparasel/scratch/VvsLMem-Cntxt-Conflict/Scripts/FRQ_Results/FRQ_RAG_VISION_Experiment_Results_logo.csv


# random sutff

In [32]:
import pandas as pd
df_txt = pd.read_csv('/users/aparasel/scratch/VvsLMem-Cntxt-Conflict/Scripts/FRQ_Results/FRQ_RAG_Experiment_Results.csv')
df_vis = pd.read_csv("/users/aparasel/scratch/VvsLMem-Cntxt-Conflict/Scripts/FRQ_Results/FRQ_RAG_VISION_Experiment_Results.csv")

In [37]:
df_vis = df_vis[df_vis['Category'] == 'Career_error']
df_txt = df_txt[df_txt['Category'] == 'Career_error']
for i in range(50):
    print(df_vis[df_txt['Pred'] == 'mis_label']['Response'])

22     system\nYou are a helpful assistant.\nuser\nCo...
28     system\nYou are a helpful assistant.\nuser\nCo...
29     system\nYou are a helpful assistant.\nuser\nCo...
34     system\nYou are a helpful assistant.\nuser\nCo...
64     system\nYou are a helpful assistant.\nuser\nCo...
                             ...                        
844    system\nYou are a helpful assistant.\nuser\nCo...
856    system\nYou are a helpful assistant.\nuser\nCo...
874    system\nYou are a helpful assistant.\nuser\nCo...
875    system\nYou are a helpful assistant.\nuser\nCo...
892    system\nYou are a helpful assistant.\nuser\nCo...
Name: Response, Length: 82, dtype: object
22     system\nYou are a helpful assistant.\nuser\nCo...
28     system\nYou are a helpful assistant.\nuser\nCo...
29     system\nYou are a helpful assistant.\nuser\nCo...
34     system\nYou are a helpful assistant.\nuser\nCo...
64     system\nYou are a helpful assistant.\nuser\nCo...
                             ...              

In [38]:
df_vis

,Unnamed: 0,ID,Instance,Category,Mis_Knowledge_Key,Context,Query,Response,Ground_Truth,Mis_Answer_Label,Assistant_response,Pred
4,4,1,Taylor Swift,Career_error,mis_knowledge1,"Taylor Swift was born on December 13, 1989. Ta...",What is the occupation of the entity?,system\nYou are a helpful assistant.\nuser\nCo...,singer-songwriter,novelist,\nThe occupation of the person is singer-song...,gt
5,5,1,Taylor Swift,Career_error,mis_knowledge2,"Taylor Swift was born on December 13, 1989. Ta...",What is the occupation of the entity?,system\nYou are a helpful assistant.\nuser\nCo...,singer-songwriter,chef,\nThe occupation of the person is singer-song...,gt
10,10,2,Elon Musk,Career_error,mis_knowledge1,"Elon Musk was born on June 28, 1971. Elon Musk...",What is the occupation of the entity?,system\nYou are a helpful assistant.\nuser\nCo...,"business magnate, investor, and engineer",professional athlete,\nThe occupation of the person is professiona...,mis_label
11,11,2,Elon Musk,Career_error,mis_knowledge2,"Elon Musk was born on June 28, 1971. Elon Musk...",What is the occupation of the entity?,system\nYou are a helpful assistant.\nuser\nCo...,"business magnate, investor, and engineer",chef,\nThe occupation of the person is CEO and ent...,gt
16,16,3,Cristiano Ronaldo,Career_error,mis_knowledge1,"Cristiano Ronaldo was born on February 5, 1985...",What is the occupation of the entity?,system\nYou are a helpful assistant.\nuser\nCo...,professional footballer,professional basketball player,\nThe occupation of the person is professiona...,gt
...,...,...,...,...,...,...,...,...,...,...,...,...
881,881,147,Shia LaBeouf,Career_error,mis_knowledge2,"Shia LaBeouf was born on June 11, 1986. Shia L...",What is the occupation of the entity?,system\nYou are a helpful assistant.\nuser\nCo...,"actor, filmmaker, and performance artist",architect,\nThe occupation of the person is actor.,gt
886,886,148,Chadwick Boseman,Career_error,mis_knowledge1,"Chadwick Boseman was born on November 29, 1976...",What is the occupation of the entity?,system\nYou are a helpful assistant.\nuser\nCo...,actor,chef,\nThe occupation of the person is actor.,gt
887,887,148,Chadwick Boseman,Career_error,mis_knowledge2,"Chadwick Boseman was born on November 29, 1976...",What is the occupation of the entity?,system\nYou are a helpful assistant.\nuser\nCo...,actor,software engineer,\nThe occupation of the person is actor.,gt
892,892,149,Lupita Nyong'o,Career_error,mis_knowledge1,"Lupita Nyong'o was born on March 1, 1983. Lupi...",What is the occupation of the entity?,system\nYou are a helpful assistant.\nuser\nCo...,actress,novelist,\nThe occupation of the person is Actress.,gt


In [11]:
df_txt['Pred'].value_counts()

Pred
gt           199
mis_label     82
neither       17
Name: count, dtype: int64

In [7]:
df_txt

,Unnamed: 0,ID,Instance,Category,Mis_Knowledge_Key,Context,Query,Response,Ground_Truth,Mis_Answer_Label,Assistant_response,Pred
4,4,1,Taylor Swift,Career_error,mis_knowledge1,"Taylor Swift was born on December 13, 1989. Ta...",What is the occupation of the entity?,system\nYou are a helpful assistant.\nuser\nCo...,singer-songwriter,novelist,\nThe occupation of the person is singer-song...,gt
5,5,1,Taylor Swift,Career_error,mis_knowledge2,"Taylor Swift was born on December 13, 1989. Ta...",What is the occupation of the entity?,system\nYou are a helpful assistant.\nuser\nCo...,singer-songwriter,chef,\nThe occupation of the person is singer-song...,gt
10,10,2,Elon Musk,Career_error,mis_knowledge1,"Elon Musk was born on June 28, 1971. Elon Musk...",What is the occupation of the entity?,system\nYou are a helpful assistant.\nuser\nCo...,"business magnate, investor, and engineer",professional athlete,\nThe occupation of the person is entrepreneu...,gt
11,11,2,Elon Musk,Career_error,mis_knowledge2,"Elon Musk was born on June 28, 1971. Elon Musk...",What is the occupation of the entity?,system\nYou are a helpful assistant.\nuser\nCo...,"business magnate, investor, and engineer",chef,\nThe occupation of the person is entrepreneu...,gt
16,16,3,Cristiano Ronaldo,Career_error,mis_knowledge1,"Cristiano Ronaldo was born on February 5, 1985...",What is the occupation of the entity?,system\nYou are a helpful assistant.\nuser\nCo...,professional footballer,professional basketball player,\nThe occupation of the person is professiona...,gt
...,...,...,...,...,...,...,...,...,...,...,...,...
881,881,147,Shia LaBeouf,Career_error,mis_knowledge2,"Shia LaBeouf was born on June 11, 1986. Shia L...",What is the occupation of the entity?,system\nYou are a helpful assistant.\nuser\nCo...,"actor, filmmaker, and performance artist",architect,"\nThe occupation of the person is actor, arch...",gt
886,886,148,Chadwick Boseman,Career_error,mis_knowledge1,"Chadwick Boseman was born on November 29, 1976...",What is the occupation of the entity?,system\nYou are a helpful assistant.\nuser\nCo...,actor,chef,\nThe occupation of the person is actor.,gt
887,887,148,Chadwick Boseman,Career_error,mis_knowledge2,"Chadwick Boseman was born on November 29, 1976...",What is the occupation of the entity?,system\nYou are a helpful assistant.\nuser\nCo...,actor,software engineer,\nThe occupation of the person is actor.,gt
892,892,149,Lupita Nyong'o,Career_error,mis_knowledge1,"Lupita Nyong'o was born on March 1, 1983. Lupi...",What is the occupation of the entity?,system\nYou are a helpful assistant.\nuser\nCo...,actress,novelist,\nThe occupation of the person is actress and...,mis_label


In [19]:
df = pd.read_csv("/users/aparasel/scratch/VvsLMem-Cntxt-Conflict/Scripts/Results/FRQ-Last_tok_LogitLens_Grouped.csv")
df[df['Layer'] == 27]['TopToken'].value_counts()

TopToken
a               113
an               62
singer           26
not              24
comedian          4
professional      1
martial           1
Name: count, dtype: int64

In [44]:
OUT_CSV = "/users/aparasel/scratch/VvsLMem-Cntxt-Conflict/Scripts/Results/FRQ-Last_tok_LogitLens_Grouped.csv"
df = pd.read_csv(OUT_CSV)
df = df[df['Category'] == 'Career_error']
df['Group'].value_counts() /28

FileNotFoundError: [Errno 2] No such file or directory: '/users/aparasel/scratch/VvsLMem-Cntxt-Conflict/Scripts/Results/FRQ-Last_tok_LogitLens_Grouped.csv'

# Categorize logos

In [18]:
from sentence_transformers import SentenceTransformer, util
import pandas as pd


st_model = SentenceTransformer("sentence-transformers/all-MiniLM-L6-v2")

def categorize_response(
    response: str,
    ground_truth: str,
    mis_label: str,
    *,
    min_sim: float = 0.40,   
    margin: float = 0.03,    
) -> str:
    response = "" if pd.isna(response) else str(response).strip()
    ground_truth = "" if pd.isna(ground_truth) else str(ground_truth).strip()
    mis_label = "" if pd.isna(mis_label) else str(mis_label).strip()

    response = response.split('person')[-1]

    if not response or (not ground_truth and not mis_label):
        return "neither"


    emb = st_model.encode([response, ground_truth, mis_label],
                          convert_to_tensor=True,
                          normalize_embeddings=True)
    emb_resp, emb_gt, emb_mis = emb[0], emb[1], emb[2]

    score_gt = util.dot_score(emb_resp, emb_gt).item()
    score_mis = util.dot_score(emb_resp, emb_mis).item()

    best = max(score_gt, score_mis)
    if best < min_sim:
        return "neither"

    if abs(score_gt - score_mis) < margin:
        return "neither"

    return "gt" if score_gt > score_mis else "mis_label"


UPDATE_CSV = '/users/aparasel/scratch/VvsLMem-Cntxt-Conflict/Scripts/FRQ_Results/FRQ_RAG_VISION_Experiment_Results_logo.csv'
df = pd.read_csv(UPDATE_CSV)
df['Assistant_response'] = df['Response'].apply(lambda x: x.split('assistant')[-1])
df['Pred'] = df.apply(lambda x: categorize_response(response=x['Assistant_response'],ground_truth=x['Ground_Truth'],mis_label=x['Mis_Answer_Label']),axis=1,)
df.to_csv(UPDATE_CSV)

In [19]:
pd.read_csv("/users/aparasel/scratch/VvsLMem-Cntxt-Conflict/Scripts/FRQ_Results/FRQ_RAG_VISION_Experiment_Results_logo.csv")

#df['Pred'].value_counts()

,Unnamed: 0,ID,Instance,Category,Mis_Knowledge_Key,Context,Query,Response,Ground_Truth,Mis_Answer_Label,Assistant_response,Pred
0,0,1,Nike,time_error,mis_knowledge1,Nike was created in 1972. The creator of Nike ...,When was the brand associated with the logo es...,system\nYou are a helpful assistant.\nuser\nCo...,1964,1972,\nThe company associated with the entity was e...,mis_label
1,1,1,Nike,time_error,mis_knowledge2,Nike was created in 1958. The creator of Nike ...,When was the brand associated with the logo es...,system\nYou are a helpful assistant.\nuser\nCo...,1964,1958,\nThe company associated with the entity was e...,mis_label
2,2,1,Nike,creator_error,mis_knowledge1,Nike was created in 1964. The creator of Nike ...,Who established the brand corresponding to the...,system\nYou are a helpful assistant.\nuser\nCo...,Bill Bowerman and Phil Knight,Michael Jordan and LeBron James,\nThe person/people who founded this company i...,mis_label
3,3,1,Nike,creator_error,mis_knowledge2,Nike was created in 1964. The creator of Nike ...,Who established the brand corresponding to the...,system\nYou are a helpful assistant.\nuser\nCo...,Bill Bowerman and Phil Knight,Steve Jobs and Bill Gates,\nThe person/people who founded this company i...,mis_label
4,4,1,Nike,content_error,mis_knowledge1,Nike was created in 1964. The creator of Nike ...,What are the main products of this brand?,system\nYou are a helpful assistant.\nuser\nCo...,athletic footwear,gourmet coffee,\nThe main products of this brand is/are athl...,gt
...,...,...,...,...,...,...,...,...,...,...,...,...
439,439,74,Meta,time_error,mis_knowledge2,Meta was created in 2006. The creator of Meta ...,When was the brand associated with the logo es...,system\nYou are a helpful assistant.\nuser\nCo...,2004,2006,\nThe company associated with the entity was e...,neither
440,440,74,Meta,creator_error,mis_knowledge1,Meta was created in 2004. The creator of Meta ...,Who established the brand corresponding to the...,system\nYou are a helpful assistant.\nuser\nCo...,Mark Zuckerberg,Elon Musk,\nThe person/people who founded this company i...,mis_label
441,441,74,Meta,creator_error,mis_knowledge2,Meta was created in 2004. The creator of Meta ...,Who established the brand corresponding to the...,system\nYou are a helpful assistant.\nuser\nCo...,Mark Zuckerberg,Bill Gates,\nThe person/people who founded this company i...,mis_label
442,442,74,Meta,content_error,mis_knowledge1,Meta was created in 2004. The creator of Meta ...,What are the main products of this brand?,system\nYou are a helpful assistant.\nuser\nCo...,social media platforms,gourmet food products,\nThe main products of this brand is/are gour...,mis_label
